In [9]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [10]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-o90vcmob
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-o90vcmob
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=705c026ded9d12d59936efdfcda04cc707876149a7563377c31a67b34e4e4d5c
  Stored in directory: /tmp/pip-ephem-wheel-cache-mt7bjm76/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [11]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <cuda.h>

__global__ void hello()
{
   int globalID = blockIdx.x * blockDim.x + threadIdx.x; // Calcula el ID global del hilo
   if(globalID == 2047) // Sólo el último hilo imprime el mensaje especial
     printf("Hello world from thread %d of block %d. Soy Fredy y mi carnet es 201011\n", threadIdx.x, blockIdx.x);
   else
     printf("Hello world from thread %d of block %d\n", threadIdx.x, blockIdx.x);
}

int main()
{
  hello<<<2,1024>>>(); // 2 bloques, 1024 hilos por bloque
  cudaDeviceSynchronize();
  return 0;
}


In [ ]:
%%cu
#include <stdio.h>
#include <cuda.h>

__global__ void hello()
{
   int globalID = blockIdx.x * blockDim.x + threadIdx.x; // Calcula el ID global del hilo
   if(globalID == 2047) // Sólo el último hilo imprime el mensaje especial
     printf("Hello world from thread %d of block %d. Soy Fredy y mi carnet es 201011\n", threadIdx.x, blockIdx.x);
   else
     printf("Hello world from thread %d of block %d\n", threadIdx.x, blockIdx.x);
}

int main()
{
  hello<<<2,1024>>>(); // 2 bloques, 1024 hilos por bloque
  cudaDeviceSynchronize();
  return 0;
}



In [ ]:
%%cu
#include <stdio.h>
#include <cuda.h>

__global__ void hello ()
{
  int myID = ( blockIdx.z * gridDim.x * gridDim.y  +
               blockIdx.y * gridDim.x +
               blockIdx.x ) * blockDim.x +
               threadIdx.x;

  printf ("Hello world from %i\n", myID);
}

int main ()
{
  dim3 g (4, 3, 2);
  hello <<< g, 10 >>> ();
  cudaDeviceSynchronize();
  return 0;
}


In [ ]:
%%cu
/*
 ============================================================================
 Author        : G. Barlas
 Version       : 1.0
 Last modified : December 2014
 License       : Released under the GNU GPL 3.0
 Description   :
 To build use  : nvcc hello2.cu -o hello2 -arch=sm_20
 ============================================================================
 */
#include <stdio.h>
#include <cuda.h>

__global__ void hello ()
{
  // Fórmula genérica para calcular el ID global
  int myID = ( blockIdx.z * gridDim.x * gridDim.y  +
               blockIdx.y * gridDim.x +
               blockIdx.x ) * blockDim.x * blockDim.y * blockDim.z +
               threadIdx.z *  blockDim.x * blockDim.y +
               threadIdx.y * blockDim.x +
               threadIdx.x;

  printf ("Hello world from %i. [Fredy 201011]\n", myID); // Modificación aquí para agregar nombre y carné
}

int main ()
{
  dim3 g (4, 2);      // Modificación aquí
  dim3 b (32, 16);    // Modificación aquí

  hello <<< g, b >>> ();
  cudaDeviceSynchronize();  // Usando cudaDeviceSynchronize ya que cudaThreadSynchronize está obsoleto
  return 0;
}


In [ ]:
%%cu
/*
 ============================================================================
 Author        : G. Barlas
 Version       : 1.0
 Last modified : December 2014
 License       : Released under the GNU GPL 3.0
 Description   :
 To build use  : nvcc hello2.cu -o hello2 -arch=sm_20
 ============================================================================
 */
#include <stdio.h>
#include <cuda.h>

__global__ void hello ()
{
  // Fórmula genérica para calcular el ID global
  int myID = ( blockIdx.z * gridDim.x * gridDim.y  +
               blockIdx.y * gridDim.x +
               blockIdx.x ) * blockDim.x * blockDim.y * blockDim.z +
               threadIdx.z *  blockDim.x * blockDim.y +
               threadIdx.y * blockDim.x +
               threadIdx.x;

  printf ("Hello world from %i. [Fredy 201011]\n", myID); // Modificación aquí para agregar nombre y carné
}

int main ()
{
  dim3 g (4, 2);      // Modificación aquí
  dim3 b (32, 16);    // Modificación aquí

  hello <<< g, b >>> ();
  cudaDeviceSynchronize();  // Usando cudaDeviceSynchronize ya que cudaThreadSynchronize está obsoleto
  return 0;
}


In [ ]:
%%cu
#include <stdio.h>
#include <cuda.h>

// Variable global para el máximo ID
__device__ int maxGlobalID = 0;

__global__ void hello ()
{
    // Fórmula genérica para calcular el ID global
    int myID = ( blockIdx.z * gridDim.x * gridDim.y  +
               blockIdx.y * gridDim.x +
               blockIdx.x ) * blockDim.x * blockDim.y * blockDim.z +
               threadIdx.z *  blockDim.x * blockDim.y +
               threadIdx.y * blockDim.x +
               threadIdx.x;

    // Actualizar maxGlobalID de manera atómica
    atomicMax(&maxGlobalID, myID);

    printf ("Hello world from %i. [Fredy 201011]\n", myID);
}

int main ()
{
    dim3 g (4, 2);
    dim3 b (32, 16);

    hello <<< g, b >>> ();
    cudaDeviceSynchronize();  // Sincronización

    // Copiar el valor del máximo ID al host y mostrarlo
    int hostMaxID;
    cudaMemcpyFromSymbol(&hostMaxID, maxGlobalID, sizeof(int));
    printf("Max Global ID is: %d\n", hostMaxID);

    return 0;
}


In [19]:
%%cu
#include <stdio.h>
#include <cuda.h>

__device__ int maxGlobalID = 0;

__global__ void hello ()
{
    // Fórmula genérica para calcular el ID global
    int myID = ( blockIdx.z * gridDim.x * gridDim.y  +
               blockIdx.y * gridDim.x +
               blockIdx.x ) * blockDim.x * blockDim.y * blockDim.z +
               threadIdx.z *  blockDim.x * blockDim.y +
               threadIdx.y * blockDim.x +
               threadIdx.x;

    // Actualizar maxGlobalID de manera atómica
    atomicMax(&maxGlobalID, myID);

    if(myID == 99999) // Solo imprime si es el ID global máximo
    {
        printf("Hello world from the max ID: %i. [Fredy 201011]\n", myID);
        printf("Grid configuration: [%d, %d, %d]\n", gridDim.x, gridDim.y, gridDim.z);
        printf("Block configuration: [%d, %d, %d]\n", blockDim.x, blockDim.y, blockDim.z);
        printf("Block Index: [%d, %d, %d] and Thread Index: [%d, %d, %d]\n", blockIdx.x, blockIdx.y, blockIdx.z, threadIdx.x, threadIdx.y, threadIdx.z);
    }
}

int main ()
{
    dim3 g (100, 1, 1); // 100 bloques
    dim3 b (1000, 1, 1); // 1000 hilos por bloque

    hello <<< g, b >>> ();
    cudaDeviceSynchronize();

    return 0;
}


Hello world from the max ID: 99999. [Fredy 201011]
Grid configuration: [100, 1, 1]
Block configuration: [1000, 1, 1]
Block Index: [99, 0, 0] and Thread Index: [999, 0, 0]

